In [1]:
import csv
from scipy import sparse
from sklearn.cluster import KMeans
import pandas as pd
import networkx as nx
import numpy as np
import time
from scipy.sparse import *
from scipy import *
from sklearn.model_selection import RandomizedSearchCV
import itertools
from tqdm import tqdm
import sys
import matplotlib.pyplot as plt
from scipy.spatial.distance import hamming
from numpy import linalg
from matplotlib.pyplot import figure
import random
import scipy
from collections import Counter

In [2]:
random.seed(123)

In [3]:
def adj_matrix(edge_list, n, is_directed, is_weighted):
    
    A = lil_matrix((n,n), dtype=float)
    f = open(edge_list)
    reader = csv.reader(f)
    
    for line in reader:
        if is_weighted:
            A[int(line[0]),int(line[1])] = round(float(line[2]),2)
        else:
            A[int(line[0]),int(line[1])] = 1
        
        if is_directed==False:
            A[int(line[1]),int(line[0])] = A[int(line[0]),int(line[1])]  
        
    return A

In [4]:
def kmeans(k, t, A):

    kmeans = KMeans(n_clusters=k, init = 'k-means++', algorithm='auto', max_iter=t).fit(A)
    
    # id_supernode : list of id_nodes in it
    partition = {i: np.where(kmeans.labels_ == i)[0] for i in range(k)}
    
    #id_node : id_supernode it belongs to
    supernode = {}
    for k,v in partition.items():
        for el in v:
            supernode[el] = k
  
    return partition, supernode

In [5]:
def lifted_density_matrix(S, n, partition, supernode):
    
    t0 = time.time()
    S_lifted = lil_matrix((n,n), dtype=float) 
    S_grass = lil_matrix((n,n), dtype=float) 
    
    for i in tqdm(range(n)):
        for j in range(n):
            s_i = supernode[i]
            s_j = supernode[j]
            
            S_lifted[i,j] = S[s_i, s_j]
            
            if s_i != s_j :
                S_grass[i,j] = S_lifted[i,j] 
            elif i != j:
                S_grass[i,j] = S_lifted[i,j] * len(partition[s_j])/(len(partition[s_j])-1)   
            else:
                S_grass[i,j] = 0
                
    t1 = time.time()
    print('Running time A_S_lifted + Grass: %f' %(t1-t0))
    return S_lifted,S_grass 

In [6]:
def density_matrix(k, edge_list, partition, A, is_directed, is_weighted, n, supernode):
    
    t0 = time.time()
    
    S = dok_matrix((k,k))        #density matrix of the expected weights
    S_prob = dok_matrix((k,k))
    
    f = open(edge_list)
    reader = csv.reader(f)
    for el in reader:
        el[0]=int(el[0])
        el[1]=int(el[1])
        if is_weighted:
            S[supernode[el[0]], supernode[el[1]]] +=  A[el[0],el[1]]
            S_prob[supernode[el[0]], supernode[el[1]]] += 1
        else: 
            S[supernode[el[0]], supernode[el[1]]] +=  1
        if is_directed==False:
            S[supernode[el[1]], supernode[el[0]]] +=  A[el[0],el[1]]
        
    for i in range(k): 
        for j in range(k): 
            den = len(partition[i])*len(partition[j])
            S[i,j] = round(S[i,j]/den,2)
            S_prob[i,j] = round(S_prob[i,j]/den,2)
            
    t1 = time.time()
    print('Running time S (kxk): %f' %(t1-t0))
    
   # S_probs, S_probs_G = lifted_density_matrix(S_prob,n,partition,supernode)  #Lifted + Grass probability matrix
   # S, S_G = lifted_density_matrix(S,n,partition,supernode)                   #Lifted + Grass  matrix
    #return  S_probs, S_probs_G,  S, S_G
    
    if is_weighted:
        return  S, S_prob
    else:
        return S
   

In [7]:
def summ_A(edge_list, n, k, t, is_directed, is_weighted):
    
    t2 = time.time()
    A = adj_matrix(edge_list, n, is_directed, is_weighted)
    t3  = time.time()
    print('Running time adj_matrix: %f' %(t3-t2))


    #partition given by the clustering
    t6 = time.time()
    partition, supernode = kmeans(k, t, A)
    t7 = time.time()
    print('Running time kmeans: %f' %(t7-t6))

    
    if is_weighted:
        S, S_prob = density_matrix(k, edge_list, partition, A, is_directed, is_weighted, n, supernode)
        return S, S_prob, partition, supernode
    else:
        S = density_matrix(k, edge_list, partition, A, is_directed, is_weighted, n, supernode)
        return S, partition, supernode

In [8]:
# syn-trans-1 (k=100, k=250, k=350)
k = 250
S, S_prob, partition, supernode = summ_A('1k.csv', 1000, k, 20, True, True)
#S_probs, S_probs_G, S, S_G, partition, supernode = summ_A('1k_no_companies.csv', '1k', 1000, k, 20)

Running time adj_matrix: 0.053328
Running time kmeans: 4.498363
Running time S (kxk): 4.222850


In [128]:
# syn-trans-10 (k=500, k=1000, k=1500)
k = 1000
S, S_prob, partition, supernode = summ_A('10k.csv', 10000, k, 20, True, True)

Running time adj_matrix: 0.520817
Running time kmeans: 58.448911
Running time S (kxk): 68.994683


In [8]:
# FACEBOOK (k=350, k=500, k=750, k=723)
k = 500
S, partition, supernode = summ_A('fb.csv', 4039, k, 20, False, False)

Running time adj_matrix: 0.806139
Running time kmeans: 36.297059
Running time S (kxk): 19.268763


In [52]:
# LASTFM (k=500, k=750, k=1000, k=1730)
k = 750
S, partition, supernode = summ_A('lastfm.csv', 7624, k, 20, False, False)

Running time adj_matrix: 0.534232
Running time kmeans: 55.897789
Running time S (kxk): 34.368591


In [82]:
# ENRON MAIL (k=1000, k=1500, k=2000, k=6137)
k = 1500
S, partition, supernode = summ_A('en.csv', 36692, k, 20, False, False)

Running time adj_matrix: 3.694930
Running time kmeans: 619.419609
Running time S (kxk): 160.278998


In [148]:
# GNUTELLA (K = 500,750,1000)
#analysis on the largest cc
k = 750
S, partition, supernode = summ_A('gnutella.csv', 6301, k, 20, True, False)

Running time adj_matrix: 0.075833
Running time kmeans: 32.503183
Running time S (kxk): 30.603523


In [78]:
# UBUNTU (K = 350,500,750)
k = 500
S, S_prob, partition, supernode = summ_A('ubuntu.csv', 3035, k, 20, False, True)

Running time adj_matrix: 1.518842
Running time kmeans: 55.855494
Running time S (kxk): 33.270426


# Possible Worlds


In [129]:
# worlds sampling

def world(graph_type, matrix, prob_matrix):
    s = nx.from_numpy_matrix(np.matrix(matrix.A), create_using=graph_type)
    world = s.copy()
    if prob_matrix != None:
        s_prob = nx.from_numpy_matrix(np.matrix(prob_matrix.A), create_using=graph_type)
        for edge in s_prob.edges(data=True):
            sample = random.uniform(0,1)
            if edge[-1]['weight'] < sample:
                world.remove_edge(edge[0], edge[1])
        return world

    else:
        for edge in s.edges(data=True):
            sample = random.uniform(0,1)
            if edge[-1]['weight'] < sample:
                world.remove_edge(edge[0], edge[1])
    return world

In [130]:
# create the set of possible worlds

type_of_graph = nx.DiGraph
possible_worlds = []
for i in tqdm(range(100)):
    possible_worlds.append(world(type_of_graph, S, S_prob))
    #possible_worlds.append(world(type_of_graph, S, None))

100%|██████████| 100/100 [00:06<00:00, 14.48it/s]


## Single Value Queries

In [ ]:
# Number of Edges
mean = 0

for graph in possible_worlds:
    mean += len(graph.edges())
    
print('mean number of edges', round(mean/100)) 

In [26]:
# diameter for directed graphs

diameter_largest_cc = 0
diameter = 0
t1 = time.time()
for graph in tqdm(possible_worlds):
    largest_cc = max(nx.strongly_connected_components(graph), key=len)
    sub_graph = graph.subgraph(largest_cc)
    diameter_largest_cc += nx.diameter(sub_graph)
    if nx.is_strongly_connected(graph) == True:
        diameter += nx.diameter(graph)
t2 = time.time()

print('Running Time:', round(t2-t1,2))
print('diameter largest cc:', round(diameter_largest_cc/100))
print('diameter:', round(diameter/100))

100%|██████████| 100/100 [00:00<00:00, 263.42it/s]

Running Time: 0.38
diameter largest cc: 0
diameter: 0


In [14]:
# diameter for undirected graphs

diameter_largest_cc = 0
diameter = 0
count = 0
t1 = time.time()
for graph in tqdm(possible_worlds):
    largest_cc = max(nx.connected_components(graph), key=len)
    sub_graph = graph.subgraph(largest_cc)
    diameter_largest_cc += nx.diameter(sub_graph)
    if nx.is_connected(graph) == True:
        count=+1
        diameter += nx.diameter(graph)

t2 = time.time()

print('Running Time:', round(t2-t1,2))
print('diameter largest cc:', round(diameter_largest_cc/100))
print('diameter:', round(diameter/count))

100%|██████████| 100/100 [00:00<00:00, 1006.71it/s]

Running Time: 0.1
diameter largest cc: 0


ZeroDivisionError: division by zero

In [15]:
# average cluster coefficient
t1 = time.time()
avg_clustering = 0
for graph in tqdm(possible_worlds):
    avg_clustering += nx.average_clustering(graph)

t2 = time.time()

print('Running Time:', round(t2-t1,2))
print('average clustering coefficient:', round(avg_clustering/100, 3))

100%|██████████| 100/100 [02:26<00:00,  1.47s/it]

Running Time: 146.8
average clustering coefficient: 0.364


In [25]:
# connected component

number_cc = 0
size = 0
for graph in tqdm(possible_worlds):
    largest_cc = max(nx.strongly_connected_components(graph), key=len)
    size += len(largest_cc)
    number_cc += len([c for c in nx.strongly_connected_components(graph)])
    sub_graph = graph.subgraph(largest_cc)
    
    
print('Number of cc:', number_cc/100)   
print('Size largest cc:', round(size/100))

100%|██████████| 100/100 [00:01<00:00, 54.87it/s]

Number of cc: 604.78
Size largest cc: 144


## Shortest Path

In [131]:
#input

#A = adj_matrix('fb.csv', 4039, False, False)
A = adj_matrix('edge_list_10k.csv', 10000, True, True)
#A = adj_matrix('edge_list_1k.csv', 1000, True, True)
#A = adj_matrix('lastfm.csv', 7624, False, False)
#A = adj_matrix('en.csv', 36692, False, False)
#A = adj_matrix('gnutella.csv', 6301, True, False)
#A = adj_matrix('ubuntu.csv', 3035, False, True)


G = nx.from_numpy_matrix(np.matrix(A.A), create_using=nx.DiGraph)

In [ ]:
# input for Enron
Data = open('en.csv', "r")
G = nx.parse_edgelist(Data, delimiter=',', create_using=nx.Graph, nodetype=int)

In [146]:
# sample one couples of nodes 
#largest_cc = sorted(nx.weakly_connected_components(G), reverse=1)[0]
sample1 = random.sample(G.nodes(),1)[0]
sample2 = random.sample(G.nodes(), 1)[0]
sample = [sample1, sample2]

In [147]:
def shortest_distance(G,start,w):
    
    shortestDistance = float("inf")
    for n in G.neighbors(source): 
        currentDistance = nx.shortest_path_length(s, source=n, target=start, weight= w, method='dijkstra')
        if currentDistance < shortestDistance:
            target = n
        shortestDistance = min(currentDistance, shortestDistance)
        
    return shortestDistance+1 if w==None else shortestDistance+S[start,n]

In [148]:
def shortest_distance(G,start,w):
    
    shortestDistance = float("inf")
    for n in G.neighbors(start): 
        currentDistance = nx.shortest_path_length(G, source=n, target=start, weight= w, method='dijkstra')
        if currentDistance < shortestDistance:
            target = n
        shortestDistance = min(currentDistance, shortestDistance)
        
    return shortestDistance+1 if w==None else shortestDistance+S[start,n]

In [149]:
def shortest_path(s, couple,w):  
    
    #shortest path for the summary
    s_i = supernode[couple[0]]
    s_j = supernode[couple[1]]
    if s_i == s_j:
        if S[s_i, s_j] != 0:
            spS = 1
        else:
            spS = shortest_distance(s, s_i, w)

    else:
         spS = nx.shortest_path_length(s, source=s_i, target=s_j, method='dijkstra', weight=w)
       
    return spS

In [155]:
#shortest path for the input
spG = nx.shortest_path_length(G, source=sample[0], target=sample[1], method='dijkstra', weight=None)

spS = []
for graph in tqdm(possible_worlds):
    spS.append(shortest_path(graph,sample,None))
    

100%|██████████| 100/100 [00:00<00:00, 57852.47it/s]


In [156]:
sp_dict = {}
for k,v in Counter(spS).items():
    sp_dict[k] = v/100

In [157]:
sp_dict

{1: 1.0}

In [158]:
spG

5

In [159]:
spS = 1
print('Average Error:',abs(spG-spS))
print('Percentage Err:',abs(spG-spS)/spG)

Average Error: 4
Percentage Err: 0.8


## Set Queries

In [42]:
#input

# = adj_matrix('fb.csv', 4039, False, False)
#A = adj_matrix('edge_list_10k.csv', 10000, True, True)
#A = adj_matrix('edge_list_1k.csv', 1000, True, True)
# = adj_matrix('lastfm.csv', 7624, False, False)
#A = adj_matrix('en.csv', 36692, False, False)
A = adj_matrix('gnutella.csv', 6301, True, False)
#A = adj_matrix('ubuntu.csv', 3035, False, True)


G = nx.from_numpy_matrix(np.matrix(A.A), create_using=type_of_graph)

In [11]:
# input for Enron
Data = open('en.csv', "r")
G = nx.parse_edgelist(Data, delimiter=',', create_using=nx.Graph,nodetype=int)

In [43]:
def precision(k, l1, l2):
    l3 = [value for value in l1 if value in l2]
    intersect = len(l3)
    return round(intersect/k,2)

In [44]:
def centrality_query(centrality_type, possible_world, k, partition, top):
    sets = []
    for graph in tqdm(possible_worlds):
        centrality = centrality_type(graph, weight=None)
        
        if centrality_type == nx.pagerank:
            for i in range(k): #normalization for pagerank
                     centrality[i] = centrality[i]/len(partition[i])
       
        if centrality_type == nx.degree_centrality:
            for i in range(k):       #normalization for the degree centrality
                if graph.has_edge(i,i) == True:
                    centrality[i] = centrality[i] + (len(partition[i]) - 1)

        
        projection = {}
        for k,v in centrality.items():
            for el in partition[k]:
                projection[el] = v

        th = sorted(list(projection.values()), reverse=True)[top]
        k2 = len(graph.nodes())
        for idx,el in enumerate(sorted(list(projection.values()), reverse=True)):
            if el<th:
                k2 = idx
                break
        answer = sorted(projection, key=projection.get, reverse=True)[:k2]
        sets.append(answer)
    return sets

In [ ]:
centrality_type = nx.eigenvector_centrality
t1 = time.time()
answers = centrality_query(centrality_type, possible_worlds, k, partition, 99)
t2 = time.time()

print('Running Time:', round(t2-t1,2))

In [49]:
#centrality query using intersection

k1 = 200
final_answer = set.intersection(*map(set,answers))
centralityG = centrality_type(G)
topG = sorted(centralityG, key=centralityG.get, reverse=True)[:k1]



# measures
r = precision(k1,topG, final_answer)
p = precision(len(final_answer), topG, final_answer)
print('Precision:', p)
print('Recall:', r)
print('f1 measure', 2*p*r/(p+r))

Precision: 0.21
Recall: 0.12
f1 measure 0.15272727272727274


In [50]:
#centrality query using frequency score

#k1 = 200

#centralityG = centrality_type(G)
#topG = sorted(centralityG, key=centralityG.get, reverse=True)[:k1]

flat_answers = [item for sublist in answers for item in sublist]
occurrences = Counter(flat_answers)
#th = max(set(occurrences.values()))
th = sorted(occurrences.values(), reverse=True)[k1]
final_answer = [k for k,v in occurrences.items() if v>=th]

# measures
r = precision(k1, topG, final_answer)
p = precision(len(final_answer), topG, final_answer)
print('Precision:', p)
print('Recall:', r)
print('f1 measure', 2*p*r/(p+r))

Precision: 0.17
Recall: 0.17
f1 measure 0.17


In [50]:
#inner most core
def inner_most_core(possible_world, k, partition):

    sets = []
    for graph in tqdm(possible_world):
        graph.remove_edges_from(nx.selfloop_edges(graph))
        coreS = nx.core_number(graph)
        
        coreS_dict = {}
        for k, v in coreS.items():
            for node in partition[k]:
                coreS_dict[node] = v 
    
        topSinG = [k for k in coreS_dict.keys() if coreS_dict[k]==max(coreS_dict.values())]

        sets.append(topSinG)
    return sets

In [52]:
t1 = time.time()
answers = inner_most_core(possible_worlds, k, partition)
t2 = time.time()

print('Running Time:', round(t2-t1,2))

100%|██████████| 100/100 [46:59<00:00, 28.20s/it]

Running Time: 2819.6


In [ ]:
#inner most core queries using intersection


final_answer = set.intersection(*map(set,answers))
coreG = nx.core_number(G)
topG = [k for k in coreG.keys() if coreG[k]==max(coreG.values())]

# measures
r = precision(len(topG),topG,final_answer)
p = precision(len(final_answer),topG, final_answer)
print('Precision:', p)
print('Recall:', r)
print('f1 measure', 2*p*r/(p+r))

In [67]:
#inner most core queries using frequency score

flat_answers = [item for sublist in answers for item in sublist]
occurrences = Counter(flat_answers)
th = sorted(occurrences.values(), reverse=True)[k1]
final_answer = [k for k,v in occurrences.items() if v>=th]



# measures
r = precision(len(topG),topG,final_answer)
p = precision(len(final_answer),topG, final_answer)
print('Precision:', p)
print('Recall:', r)
print('f1 measure', round(2*p*r/(p+r),2))

Precision: 1.0
Recall: 0.99
f1 measure 0.99
